In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import scipy.io
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
exp_data = scipy.io.loadmat('/content/drive/My Drive/experimental_traces.mat')

In [ ]:
traces = exp_data['exp_data']
time = exp_data['time']

In [ ]:
def Intramonomer_transfer(powers1,powers, powersB,hopP700, bulkFluor, hopRed,hopIntra, iter, probHop, trap_prob ):

  dt = 0.01 #time step in ps
  t = np.arange(0,10,dt)  #time range in ps
  max_q = iter
  max_i = len(t)
  max_p = 4


  annihilationvec = np.zeros((max_q,max_i))
  RCtrapvec = np.zeros((max_q,max_i))
  totBulk = np.zeros((max_q,max_i))
  totRed = np.zeros((max_q,max_i))
  Red_results = np.zeros((max_i,max_p))
  Bulk_results = np.zeros((max_i,max_p))

  cols = ['black','red','cyan','green'];
  for p in range(0,4):

    monomer =  np.array([1,4,4,0,2,2,2,2,2,2])

        # 4 is proximal red Chl
        # 1 is distal red Chl
        # 2 is bulk Chl


    #find grid locations of each species
    red = np.array(np.where( monomer == 4))
    blanks = np.array(np.where (monomer==0))
    allpigments = np.array(np.where(monomer!=0))
    red_1 = np.array(np.where(monomer==1))
    bulk = np.array(np.where(monomer==2))

    power_R = (powers[p]/(np.shape(red)[1]) )#excitations per pigment
    power_1 = (powers1[p]/np.shape(red_1)[1]);
    power_B = (powersB[p]/np.shape(bulk)[1]);

    for q in range(0,iter):

      array = monomer*0 #start with no excitations
      array[blanks]=-1

  #array is randomly populated with excitations

      dist =np.random.rand(1,np.shape(red)[1])
      array[red[np.where(dist<(power_R))]]=1

      dist1 =np.random.rand(1,np.shape(red_1)[1])
      array[red_1[np.where(dist1<(power_1))]]=1

      dist2 = np.random.rand(1,np.shape(bulk)[1])
      array[bulk[np.where(dist2<(power_B))]]=1



      totexcited = np.sum(np.sum(array>0))
      foo=[]
      annihilation=0
      RCtrap=0
      fluortot=0

      for i in range(0,len(t)):

        ind=np.where(array==1)
        ind = np.array(ind)
        dist=np.random.rand(1,np.shape(ind)[1])

        hoppers = np.array(np.where(dist<probHop))
        hoppers = hoppers[1]
        for elem in range(0,np.shape(hoppers)[0]):
          nn = np.zeros((3,1))
          tup = ind[0][hoppers[elem]]
          if (monomer[tup]==2 )| (monomer[tup]==4):
            if monomer[tup]==2:
              nn[0]=0
              nn[1]=1
              nn[2]=2
            elif monomer[tup]==4:
              if tup==1:
                nn[0]=2
              else:
                nn[0]=1

            nn[np.array(np.nonzero(nn)).transpose()[:,0]]
            nnvectype = np.zeros((len(nn),1))
            for k in range(0,len(nn)):
              ind1 = int(nn[k])
              nnvectype[k]=np.sum(monomer[ind1]==2)*2
              nnvectype[k]=nnvectype[k]+np.sum(monomer[ind1]==4)*4
              nnvectype[k]=nnvectype[k]+np.sum(monomer[ind1]==1)
              nnvectype[k]=nnvectype[k]+np.sum(monomer[ind1]==0)*5

            probs = np.zeros(np.shape(nn))
            if monomer[tup]==2:  # if excitation is on bulk  Chl
              probs[np.array(np.where(nnvectype==1))]=0
              probs[np.array(np.where(nnvectype==2))]=0
              probs[np.array(np.where(nnvectype==3))]=dt/hopRed
              probs[np.array(np.where(nnvectype==4))]=dt/hopRed
              probs[np.array(np.where(nnvectype==5))]=0

              foo=np.cumsum(probs)
              probs=foo*np.ceil(probs).transpose()
              if dist[0][hoppers[elem]]<np.max(probs):
                hop=np.where(np.random.rand(1)*np.max(probs)<probs)
                hop = hop[1][0]
                array[tup]=0 #deexcite the complex you are on
                array[int(nn[hop])]=array[int(nn[hop])]+1 #excite the hop

            if (monomer[tup]==4 )| (monomer[tup] == 3): # if excitation is on a proximal red chl or P700
              probs[np.array(np.where(nnvectype==1))]=0
              probs[np.array(np.where(nnvectype==2))]=0
              probs[np.array(np.where(nnvectype==3))]=dt/hopIntra
              probs[np.array(np.where(nnvectype==4))]=dt/hopIntra
              probs[np.array(np.where(nnvectype==5))]=0

              foo=np.cumsum(probs)
              probs=foo*np.ceil(probs).transpose()
              if dist[0][hoppers[elem]]<np.max(probs):
                hop=np.where(np.random.rand(1)*np.max(probs)<probs)
                hop = hop[1][0]
                array[tup]=0 #deexcite the complex you are on
                array[int(nn[hop])]=array[int(nn[hop])]+1 #excite the hop

            del nnvectype
            del probs
            del nn

        annihilation=annihilation+np.sum(array[np.where(array>1)])-np.shape(np.where(array>1))[1] # determine if there is annihilation

        # determine if there is fluorescence
        ind=np.array(np.where(array>0)) #Find all the excitations
        dist=np.random.rand(1,np.shape(ind)[1]) # determine if they fluoresce
        fluortot=fluortot+np.shape(np.where(dist<(dt/bulkFluor)))[1]
        array[ind[np.where(dist<(dt/bulkFluor))]]=0

        # determine if there is transfer to P700
        ind=np.array(np.where(array>0)) # find the excitations
        dist=np.random.rand(1,np.shape(ind)[1]) # determine if they transfer
        hop_test = np.random.rand(1,np.shape(dist)[1])
        for el in range(0,np.shape(dist)[1]):
          if hop_test[0][el]<trap_prob:
            if dist[0][el]<(dt/hopP700):
              RCtrap=RCtrap+1
              array[ind[np.where(dist[0][el]<(dt/hopP700))]]=0


        array[np.where(array>1)]=1
        annihilationvec[q][i]=annihilation
        RCtrapvec[q][i]=RCtrap
        totBulk[q][i]=np.sum(array[bulk]>0)
        totRed[q][i]=np.sum(array[red]>0)+np.sum(array[red_1]>0)


    temp=np.mean(totRed,axis=0)
    temp2=np.mean(totBulk,axis=0);
    Red_results[:,p]=np.mean(totRed,axis=0)
    Bulk_results[:,p]=np.mean(totBulk,axis=0)

    plt.plot(t*1000,Red_results[:,p]/Red_results[40,p],color=cols[p])
    plt.plot(t*1000,Bulk_results[:,p],color=cols[p],linestyle='dashed')

  plt.plot(time[0],traces[33:,0])
  plt.plot(time[0],traces[33:,1])
  plt.plot(time[0],traces[33:,2])
  plt.plot(time[0],traces[33:,3])
  plt.show()
  return Red_results



In [ ]:
Results = Intramonomer_transfer(0.33*np.array([0.4025, 0.762, 1.438, 2.286]),0.67*np.array([0.4025, 0.762, 1.438, 2.286]),1*np.array([0.4025, 0.762, 1.438, 2.286]),70,5000,0.15,3.2,5000,1,1)

In [ ]:
dt = 0.01 #time step in ps
t = np.arange(0,10,dt)
cols = ['black','red','cyan','green']

for p in range(0,4):
  plt.plot(t*1000,Test_results2[:,p]/Test_results2[40,p],color=cols[p])
  plt.plot(time[0],traces[33:,p])

plt.xlim((0,10000))
plt.ylim((0.3,1.1))